In this Jupyter Notebook, we will be re-writing the implementation of Uniswap V3 in Python.

In [2]:
import numpy as np
import pandas as pd
import math
from sympy import symbols, Symbol, Eq, nsolve, solve, sqrt
from decimal import Decimal, getcontext

getcontext().prec = 100

# Variables

Here, we define our variables.

In [3]:
"""
The current_state variable is a dictionary that maps Initiated ticks to another dictionary, which gives properties of the liquidity pool at that tick:
1) Net liquidity at each tick
2) Gross liquidity at each tick

An example will be:
{'tick = 1':{'net_liquidity':1, 'gross_liquidity':1},
 'tick = 3':{'net_liquidity':3, 'gross_liquidity':3}}
"""

current_state = {}

"""
The current_spot_price is a variable that gives the current swap rate of the pool (in terms of token1/token0)
"""

current_spot_price = 0

"""
The lp_dataframe stores a record of how much liquidity and at what range liquidity providers current have in the pool
"""

lp_dataframe = pd.DataFrame(columns = ['address','tick_lower','tick_upper','L'])

"""
The fees_collected variable is a dictionary that keeps track of the fee earned by every LPs. Key = Address, Value = [Token 0 earned, Token 1 earned]
A new key is appended to the dictionary whenever a new LP provides liquidity.
"""

fees_collected = {}

# Functions

Here, we write functions that will be useful. All functions are written in relation to the UniSwap V3 protocol.

### Get Tick

This returns the tick that corresponds to a given price.

In [4]:
"""
Parameters:
_price_ (float): this is the price

Results:
tick (int): this is the tick that corresponds to the given price
"""

def Get_Tick(_price_):
    return math.floor(math.log(_price_,1.0001))

### Get Price

This returns the price that corresponds to a given tick.

In [5]:
"""
Parameters:
_tick_ (int): this is the tick

Results:
price (float): this is the price that corresponds to the given tick

"""

def Get_Price(_tick_):
    return Decimal(str(1.0001**_tick_))

### Get Total Liquidity

This returns the total liquidity of the pool at any ticks.

In [6]:
"""
Parameters:
_current_state_ (dict): this gives the current state of the pool
_tick_ (int): this is the tick

Results:
total_liquidity (float): this gives total liquidity available at the given tick
"""

def Get_Total_Liquidity(_current_state_, _tick_):

    # Get all initiated ticks less than _tick_
    all_ticks_before = [ticks for ticks in list(_current_state_.keys()) if ticks <= _tick_]

    # Run a loop to sum all the net liquidity to get total liquidity
    total_liquidity = 0

    for ticks in all_ticks_before:
        total_liquidity = total_liquidity + _current_state_[ticks]['net_liquidity']
    
    return Decimal(str(total_liquidity))

### Get Value From Liquidity

This returns the value (in terms of token1) hold by a LP, given the current contract price and market price.

In [ ]:
def Get_Value_From_Liquidity(_L_, _price_lower_, _price_upper_, _current_contract_price_, _current_market_price):

    # Case if current contract price is below liquidity provision range
    if _current_contract_price_ <= _price_lower_:

        real_balance_token0 = _L_*(1/_price_lower_.sqrt() - 1/_price_upper_.sqrt())
        value = real_balance_token0*_current_market_price

        return value
    
    # Case if current contract price is above liquidity provision range
    elif _current_contract_price_ >= _price_upper_:

        real_balance_token1 = _L_*(_price_upper_.sqrt() - _price_lower_.sqrt())
        value = real_balance_token1

        return value
    
    # Case if current contract price is within liquidity provision range
    else:
        real_balance_token0 = _L_*(1/_current_contract_price_.sqrt() - 1/_price_upper_.sqrt())
        real_balance_token1 = _L_*(_current_contract_price_.sqrt() - _price_lower_.sqrt())
        value = real_balance_token0*_current_market_price + real_balance_token1

        return value


### Provide Liquidity

This function is used to add liquidity to the liquidity pool.

In [8]:
def Provide_Liquidity(_current_state_, _current_spot_price_, _lp_dataframe_, _fees_collected_, _address_, _price_lower_, _price_upper_, _L_):

    """
    Initial Calculations
    """
    # First, get prices that corresponds to _tick_lower_ and _tick_upper_
    tick_lower = Get_Tick(_price_lower_)
    tick_upper = Get_Tick(_price_upper_)
    L = _L_

    """
    Liquidity Provision
    """
    # For liquidity provision, we split into 2 cases:

    # Case 1) Liquidity pool is empty
    if not bool(_current_state_.keys()):
        
        _current_state_[tick_lower] = {'net_liquidity':L, 'gross_liquidity':L}
        _current_state_[tick_upper] = {'net_liquidity':-L, 'gross_liquidity':L}
        _lp_dataframe_.loc[len(_lp_dataframe_)] = [_address_, tick_lower, tick_upper, L]
        _fees_collected_[_address_] = [0,0]

        #print("System Message: Trader " + _address_ + " sends liquidity " + str(L) + " into the pool from price " + str(_price_lower_) + " to " + str(_price_upper_))

        # Return current_state and all function below do not need to be run
        return _current_state_, _current_spot_price_, _lp_dataframe_, _fees_collected_
    

    else:
        
        # Case 2) Liquidity pool is not empty
        
        # If tick_lower provided is not already initiated, we initiate the tick_lower
        if tick_lower not in _current_state_.keys():

            # Add the new initiated tick and corresponding information
            _current_state_[tick_lower] = {'net_liquidity':L, 'gross_liquidity':L}

            # Sort dictionary by keys
            _current_state_ = dict(sorted(_current_state_.items()))

        # If tick_lower is already initiated, we can simply sum the net liquidity, gross liquidity and net real reserves
        else:

            _current_state_[tick_lower]['net_liquidity'] = _current_state_[tick_lower]['net_liquidity'] + L
            _current_state_[tick_lower]['gross_liquidity'] = _current_state_[tick_lower]['gross_liquidity'] + L


        # If tick_upper provided is not already initiated, we initiate the tick_upper
        if tick_upper not in _current_state_.keys():
            
            # Add the new initiated tick and corresponding information
            _current_state_[tick_upper] = {'net_liquidity':-L, 'gross_liquidity':L}

            # Sort dictionary by keys
            _current_state_ = dict(sorted(_current_state_.items()))

        # If tick_upper is already initiated, we can simply minus the net liquidity, net real reserves and sum the gross liquidity
        else:    
            _current_state_[tick_upper]['net_liquidity'] = _current_state_[tick_upper]['net_liquidity'] - L
            _current_state_[tick_upper]['gross_liquidity'] = _current_state_[tick_upper]['gross_liquidity'] + L
            
        
        # Add address if this address hasn't previously provided liquidity
        if _address_ not in _fees_collected_.keys():
            _fees_collected_[_address_] = [0,0]

        # Lastly, add this liquidity provision to the dataframe
        _lp_dataframe_.loc[len(_lp_dataframe_)] = [_address_, tick_lower, tick_upper, L]
        _lp_dataframe_ = _lp_dataframe_.groupby(['address','tick_lower','tick_upper']).sum().reset_index()

        #print("System Message: Trader " + _address_ + " sends liquidity " + str(L) + " into the pool from price " + str(_price_lower_) + " to " + str(_price_upper_))

        """
        Returns
        """
        # Return the new _current_state_
        return _current_state_, _current_spot_price_, _lp_dataframe_, _fees_collected_


### Swap 0 to 1

This function is used to swap token 0 to token 1, specifying the amount of token 0.

In [10]:
def Swap_Token0_to_Token1(_current_state_, _current_spot_price_, _lp_dataframe_, _fees_collected_,
                          _token0_amount_, _fees_):
    
    # Initial calculations
    _current_spot_price_ = Decimal(str(_current_spot_price_))
    current_tick = Get_Tick(_current_spot_price_)
    closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x < current_tick])
    closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x >= current_tick])
    price_lower = Get_Price(closest_lower_initiated_ticks)
    price_upper = Get_Price(closest_upper_initiated_ticks)
    current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
    
    real_balance_token0 = current_liquidity/(_current_spot_price_.sqrt()) - current_liquidity/(price_upper.sqrt())
    real_balance_token1 = current_liquidity*(_current_spot_price_.sqrt()) - current_liquidity*(price_lower.sqrt())

    max_token0_trade_in_this_range = current_liquidity/(price_lower.sqrt()) - current_liquidity/(price_upper.sqrt())

    # Check whether if this swap can be executed within the current initated tick range
    if (1-_fees_)*_token0_amount_ <= max_token0_trade_in_this_range - real_balance_token0:

        """
        If trade CAN be completed in current interval
        """
        
        # Calculate fees earned by each LP
        fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
        fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
        fees_percentage['L'] = fees_percentage['L']*_fees_*_token0_amount_
        for i in np.arange(len(fees_percentage)):
            _fees_collected_[fees_percentage['address'].iloc[i]][0] = _fees_collected_[fees_percentage['address'].iloc[i]][0] + fees_percentage['L'].iloc[i]

        # Calculate new amount of token 0
        real_balance_token0 = Decimal(float(real_balance_token0) + (1-_fees_)*_token0_amount_)
        # Solve for new amount of token 1
        token1_to_trader = real_balance_token1 - ((current_liquidity**2)/(real_balance_token0 + current_liquidity/(price_upper.sqrt())) - current_liquidity*(price_lower.sqrt()))
        real_balance_token1 = real_balance_token1 - token1_to_trader

        _new_spot_price_ = (real_balance_token1 + current_liquidity*(price_lower.sqrt()))/(real_balance_token0 + current_liquidity/(price_upper.sqrt()))

        return _current_state_, _new_spot_price_, _fees_collected_, token1_to_trader
    
    else:

        """
        If trade CANNOT be completed in current interval
        """
        
        # This variable records the total amount of token 1 that will be sent to the trader
        total_token1_to_trader = 0

        # Do the same thing until _token0_amount_ is can finish executing in a final range of initiated ticks
        while (1-_fees_)*_token0_amount_ > max_token0_trade_in_this_range - real_balance_token0:

            # Calculate fees earned by each LP
            fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
            fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
            fees_percentage['L'] = fees_percentage['L']*_fees_*float(max_token0_trade_in_this_range - real_balance_token0)
            for i in np.arange(len(fees_percentage)):
                _fees_collected_[fees_percentage['address'].iloc[i]][0] = _fees_collected_[fees_percentage['address'].iloc[i]][0] + fees_percentage['L'].iloc[i]

            # Update _token0_amount_
            _token0_amount_ = _token0_amount_ - (1/(1-_fees_))*float(max_token0_trade_in_this_range - real_balance_token0)
            
            # First, we do the swap for the current range where the spot price lies
            real_balance_token0 =  max_token0_trade_in_this_range
            token1_to_trader = real_balance_token1 - ((current_liquidity**2)/(real_balance_token0 + current_liquidity/(price_upper.sqrt())) - current_liquidity*(price_lower.sqrt()))
            real_balance_token1 = real_balance_token1 - token1_to_trader

            # Save the amount of token 0 that is swapped within this range
            total_token1_to_trader = total_token1_to_trader + token1_to_trader

            # Update environmental variables
            closest_upper_initiated_ticks = closest_lower_initiated_ticks
            closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x < closest_upper_initiated_ticks])
            price_lower = Get_Price(closest_lower_initiated_ticks)
            price_upper = Get_Price(closest_upper_initiated_ticks)
            current_spot_price = Get_Price(closest_upper_initiated_ticks)
            current_spot_price = Decimal(str(current_spot_price))
            current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
            real_balance_token0 = current_liquidity/(current_spot_price.sqrt()) - current_liquidity/(price_upper.sqrt())
            real_balance_token1 = current_liquidity*(current_spot_price.sqrt()) - current_liquidity*(price_lower.sqrt())
            max_token0_trade_in_this_range = current_liquidity*(price_upper.sqrt()) - current_liquidity*(price_lower.sqrt())


        # At the last range of initiated where the swap can be completed
        fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
        fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
        fees_percentage['L'] = fees_percentage['L']*_fees_*_token0_amount_
        for i in np.arange(len(fees_percentage)):
            _fees_collected_[fees_percentage['address'].iloc[i]][0] = _fees_collected_[fees_percentage['address'].iloc[i]][0] + fees_percentage['L'].iloc[i]

        real_balance_token0 = Decimal(float(real_balance_token0) + (1-_fees_)*_token0_amount_)
        token1_to_trader = real_balance_token1 - ((current_liquidity**2)/(real_balance_token0 + current_liquidity/(price_upper.sqrt())) - current_liquidity*(price_lower.sqrt()))
        real_balance_token1 = real_balance_token1 - token1_to_trader

        total_token1_to_trader = total_token1_to_trader + token1_to_trader

        _new_spot_price_ = (real_balance_token1 + current_liquidity*(price_lower.sqrt()))/(real_balance_token0 + current_liquidity/(price_upper.sqrt()))

        return _current_state_, _new_spot_price_, _fees_collected_, total_token1_to_trader
    


### Swap 1 to 0

This function is used to swap token 1 to token 0, specifying the amount of token 1.

In [11]:
def Swap_Token1_to_Token0(_current_state_, _current_spot_price_, _lp_dataframe_, _fees_collected_,
                          _token1_amount_, _fees_):
    
    # Initial calculations
    _current_spot_price_ = Decimal(str(_current_spot_price_))
    current_tick = Get_Tick(_current_spot_price_)
    closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x <= current_tick])
    closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x > current_tick])
    price_lower = Get_Price(closest_lower_initiated_ticks)
    price_upper = Get_Price(closest_upper_initiated_ticks)
    current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
    
    real_balance_token0 = current_liquidity/(_current_spot_price_.sqrt()) - current_liquidity/(price_upper.sqrt())
    real_balance_token1 = current_liquidity*(_current_spot_price_.sqrt()) - current_liquidity*(price_lower.sqrt())

    max_token1_trade_in_this_range = current_liquidity*(price_upper.sqrt()) - current_liquidity*(price_lower.sqrt())   

    # Check whether if this swap can be executed within the current initated tick range
    if (1-_fees_)*_token1_amount_ <= max_token1_trade_in_this_range - real_balance_token1:

        """
        If trade CAN be completed in current interval
        """

        # Calculate fees earned by each LP
        fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
        fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
        fees_percentage['L'] = fees_percentage['L']*_fees_*_token1_amount_
        for i in np.arange(len(fees_percentage)):
            _fees_collected_[fees_percentage['address'].iloc[i]][1] = _fees_collected_[fees_percentage['address'].iloc[i]][1] + fees_percentage['L'].iloc[i]
        
        # Calculate new amount of token 1
        real_balance_token1 = Decimal(float(real_balance_token1) + (1-_fees_)*_token1_amount_)
        # Solve for new amount of token 0
        token0_to_trader = real_balance_token0 - ((current_liquidity**2)/(real_balance_token1 + current_liquidity*(price_lower.sqrt())) - current_liquidity/(price_upper.sqrt()))
        real_balance_token0 = real_balance_token0 - token0_to_trader

        _new_spot_price_ = (real_balance_token1 + current_liquidity*(price_lower.sqrt()))/(real_balance_token0 + current_liquidity/(price_upper.sqrt()))

        return _current_state_, _new_spot_price_, _fees_collected_, token0_to_trader
    
    else:

        """
        If trade CANNOT be completed in current interval
        """
        
        # This variable records the total amount of token 0 that will be sent to the trader
        total_token0_to_trader = 0

        # Do the same thing until _token1_amount_ can finish executing in a final range of initiated ticks
        while (1-_fees_)*_token1_amount_ > max_token1_trade_in_this_range - real_balance_token1:

            # Calculate fees earned by each LP
            fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
            fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
            fees_percentage['L'] = fees_percentage['L']*_fees_*float(max_token1_trade_in_this_range - real_balance_token1)
            for i in np.arange(len(fees_percentage)):
                _fees_collected_[fees_percentage['address'].iloc[i]][1] = _fees_collected_[fees_percentage['address'].iloc[i]][1] + fees_percentage['L'].iloc[i]

            # Update _token1_amount_
            _token1_amount_ = _token1_amount_ - (1/(1-_fees_))*float(max_token1_trade_in_this_range - real_balance_token1)
            
            # First, we do the swap for the current range where the spot price lies
            real_balance_token1 =  max_token1_trade_in_this_range
            token0_to_trader = real_balance_token0 - ((current_liquidity**2)/(real_balance_token1 + current_liquidity*(price_lower.sqrt())) - current_liquidity/(price_upper.sqrt()))
            real_balance_token0 = real_balance_token0 - token0_to_trader

            # Save the amount of token 0 that is swapped within this range
            total_token0_to_trader = total_token0_to_trader + token0_to_trader

            # Update environmental variables
            closest_lower_initiated_ticks = closest_upper_initiated_ticks
            closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x > closest_lower_initiated_ticks])
            price_lower = Get_Price(closest_lower_initiated_ticks)
            price_upper = Get_Price(closest_upper_initiated_ticks)
            current_spot_price = Get_Price(closest_lower_initiated_ticks)
            current_spot_price = Decimal(str(current_spot_price))
            current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
            real_balance_token0 = current_liquidity/(current_spot_price.sqrt()) - current_liquidity/(price_upper.sqrt())
            real_balance_token1 = current_liquidity*(current_spot_price.sqrt()) - current_liquidity*(price_lower.sqrt())
            max_token1_trade_in_this_range = current_liquidity/(price_lower.sqrt()) - current_liquidity/(price_upper.sqrt())


        # At the last range of initiated where the swap can be completed
        fees_percentage = _lp_dataframe_[(_lp_dataframe_['tick_lower'] <= closest_lower_initiated_ticks) & (_lp_dataframe_['tick_upper'] >= closest_upper_initiated_ticks)][['address','L']]
        fees_percentage['L'] = fees_percentage['L']/fees_percentage['L'].sum()
        fees_percentage['L'] = fees_percentage['L']*_fees_*_token1_amount_
        for i in np.arange(len(fees_percentage)):
            _fees_collected_[fees_percentage['address'].iloc[i]][1] = _fees_collected_[fees_percentage['address'].iloc[i]][1] + fees_percentage['L'].iloc[i]

        real_balance_token1 = Decimal(float(real_balance_token1) + (1-_fees_)*_token1_amount_)
        token0_to_trader = real_balance_token0 - ((current_liquidity**2)/(real_balance_token1 + current_liquidity*(price_lower.sqrt())) - current_liquidity/(price_upper.sqrt()))
        real_balance_token0 = real_balance_token0 - token0_to_trader

        total_token0_to_trader = total_token0_to_trader + token0_to_trader

        _new_spot_price_ = (real_balance_token1 + current_liquidity*(price_lower.sqrt()))/(real_balance_token0 + current_liquidity/(price_upper.sqrt()))

        return _current_state_, _new_spot_price_, _fees_collected_, total_token0_to_trader




# Search Quantity Function

The two function below is mainly used to search for quantity needed to be traded in order for the price to reach a certain number.

In [1]:
def Get_Token_By_Price_Impact(_current_state_, _current_spot_price_, _final_spot_price_):
    
    # Get info on _current_spot_price_
    current_tick = Get_Tick(_current_spot_price_)
    closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x <= current_tick])
    closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x > current_tick])

    # Get info on _final_spot_price_
    final_tick = Get_Tick(_final_spot_price_)


    if _final_spot_price_ == _current_spot_price_:

        raise TypeError("Invalid price impact, price imapct is 0")


    # If price impact is positive
    elif final_tick > current_tick:

        if final_tick <= closest_upper_initiated_ticks:

            """
            If price impact can be achieve in current range
            """

            total_token1_to_trader = 0

            # Calculate the amount of token 1 left to trade in current range
            _current_spot_price_ = Decimal(str(_current_spot_price_))
            _final_spot_price_ = Decimal(str(_final_spot_price_))
            current_tick = Get_Tick(_current_spot_price_)
            closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x <= current_tick])
            closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x > current_tick])
            price_lower = Get_Price(closest_lower_initiated_ticks)
            price_upper = Get_Price(closest_upper_initiated_ticks)
            current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
            real_balance_token1 = current_liquidity*(_current_spot_price_.sqrt()) - current_liquidity*(price_lower.sqrt())
            final_balance_token1 = current_liquidity*(_final_spot_price_.sqrt()) - current_liquidity*(price_lower.sqrt())

            total_token1_to_trader  =  final_balance_token1 - real_balance_token1

            return float(total_token1_to_trader)

        else:

            """
            If price impact cannot be achieve in current range
            """

            total_token1_to_trader = 0

            # Calculate the amount of token 1 left to trade in current range
            _current_spot_price_ = Decimal(str(_current_spot_price_))
            _final_spot_price_ = Decimal(str(_final_spot_price_))
            current_tick = Get_Tick(_current_spot_price_)
            closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x <= current_tick])
            closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x > current_tick])
            price_lower = Get_Price(closest_lower_initiated_ticks)
            price_upper = Get_Price(closest_upper_initiated_ticks)
            current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
            real_balance_token1 = current_liquidity*(_current_spot_price_.sqrt()) - current_liquidity*(price_lower.sqrt())
            max_token1_trade_in_this_range = current_liquidity*(price_upper.sqrt()) - current_liquidity*(price_lower.sqrt())

            total_token1_to_trader  = total_token1_to_trader + max_token1_trade_in_this_range - real_balance_token1


            # Loop until we reach the range where final spot price lies
            closest_lower_initiated_ticks = closest_upper_initiated_ticks
            closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x > closest_lower_initiated_ticks])

            while final_tick > closest_upper_initiated_ticks:
                
                price_lower = Get_Price(closest_lower_initiated_ticks)
                price_upper = Get_Price(closest_upper_initiated_ticks)
                current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
                max_token1_trade_in_this_range = current_liquidity*(price_upper.sqrt()) - current_liquidity*(price_lower.sqrt())

                total_token1_to_trader  = total_token1_to_trader + max_token1_trade_in_this_range

                closest_lower_initiated_ticks = closest_upper_initiated_ticks
                closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x > closest_lower_initiated_ticks])
            

            # Lastly, calculate the amount of token 1 left to trade in the final range until we reach final spot price
            price_lower = Get_Price(closest_lower_initiated_ticks)
            price_upper = Get_Price(closest_upper_initiated_ticks)
            current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
            real_balance_token1 = current_liquidity*(_final_spot_price_.sqrt()) - current_liquidity*(price_lower.sqrt())

            total_token1_to_trader  = total_token1_to_trader + real_balance_token1

            return float(total_token1_to_trader)


    # If price impact is negative
    else :

        
        if final_tick >= closest_lower_initiated_ticks:

            """
            If price impact cannot be achieve in current range
            """

            total_token0_to_trader = 0

            # Calculate the amount of token 0 left to trade in current range
            _current_spot_price_ = Decimal(str(_current_spot_price_))
            _final_spot_price_ = Decimal(str(_final_spot_price_))
            current_tick = Get_Tick(_current_spot_price_)
            closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x <= current_tick])
            closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x > current_tick])
            price_lower = Get_Price(closest_lower_initiated_ticks)
            price_upper = Get_Price(closest_upper_initiated_ticks)
            current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
            real_balance_token0 = current_liquidity/(_current_spot_price_.sqrt()) - current_liquidity/(price_upper.sqrt())
            final_balance_token0 = current_liquidity/(_final_spot_price_.sqrt()) - current_liquidity/(price_upper.sqrt())

            total_token0_to_trader  = final_balance_token0 - real_balance_token0

            return float(total_token0_to_trader)

        else:

            """
            If price impact cannot be achieve in current range
            """

            total_token0_to_trader = 0

            # Calculate the amount of token 0 left to trade in current range
            _current_spot_price_ = Decimal(str(_current_spot_price_))
            _final_spot_price_ = Decimal(str(_final_spot_price_))
            current_tick = Get_Tick(_current_spot_price_)
            closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x <= current_tick])
            closest_upper_initiated_ticks = np.min([x for x in list(_current_state_.keys()) if x > current_tick])
            price_lower = Get_Price(closest_lower_initiated_ticks)
            price_upper = Get_Price(closest_upper_initiated_ticks)
            current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
            real_balance_token0 = current_liquidity/(_current_spot_price_.sqrt()) - current_liquidity/(price_upper.sqrt())
            max_token0_trade_in_this_range = current_liquidity/(price_lower.sqrt()) - current_liquidity/(price_upper.sqrt())

            total_token0_to_trader  = total_token0_to_trader + max_token0_trade_in_this_range - real_balance_token0


            # Loop until we reach the range where final spot price lies
            closest_upper_initiated_ticks = closest_lower_initiated_ticks
            closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x < closest_upper_initiated_ticks])

            while final_tick < closest_lower_initiated_ticks:
                
                price_lower = Get_Price(closest_lower_initiated_ticks)
                price_upper = Get_Price(closest_upper_initiated_ticks)
                current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
                max_token0_trade_in_this_range = current_liquidity/(price_lower.sqrt()) - current_liquidity/(price_upper.sqrt())

                total_token0_to_trader  = total_token0_to_trader + max_token0_trade_in_this_range

                closest_upper_initiated_ticks = closest_lower_initiated_ticks
                closest_lower_initiated_ticks = np.max([x for x in list(_current_state_.keys()) if x < closest_upper_initiated_ticks])
            

            # Lastly, calculate the amount of token 0 left to trade in the final range until we reach final spot price
            price_lower = Get_Price(closest_lower_initiated_ticks)
            price_upper = Get_Price(closest_upper_initiated_ticks)
            current_liquidity = Get_Total_Liquidity(_current_state_, closest_lower_initiated_ticks)
            real_balance_token0 = current_liquidity/(_final_spot_price_.sqrt()) - current_liquidity/(price_upper.sqrt())

            total_token0_to_trader  = total_token0_to_trader + real_balance_token0

            return float(total_token0_to_trader)

